In [1]:
import torch
import torchvision
import onnx
import torch.nn as nn
import torch.nn.functional as F

In [2]:
torch.manual_seed(1)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension(批大小维度)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [28]:
########### 测试BatchNorm是否会的对onnx保存产生影响 ############
########### 结论是：不会 ################
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        
        self.net = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.net(x)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension(批大小维度)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (net): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [29]:
params = list(net.parameters())
print(len(params))
print(type(params[0]))
for i in range(10):
    print("第{}个参数形状为{}".format(i,params[i].size()))

16
<class 'torch.nn.parameter.Parameter'>
第0个参数形状为torch.Size([6, 1, 5, 5])
第1个参数形状为torch.Size([6])
第2个参数形状为torch.Size([6, 1, 5, 5])
第3个参数形状为torch.Size([6])
第4个参数形状为torch.Size([6])
第5个参数形状为torch.Size([6])
第6个参数形状为torch.Size([16, 6, 5, 5])
第7个参数形状为torch.Size([16])
第8个参数形状为torch.Size([16])
第9个参数形状为torch.Size([16])


In [30]:
input = torch.randn(1, 1, 32, 32)
print(input)
print('conv1输出的形状：',net.conv1(input).size()) # conv1输出的形状

out = net(input)
print('输出层形状',out.size())
print('输出层结果：',out)

tensor([[[[ 0.7545, -0.1652, -0.0101,  ..., -0.3952, -0.0747, -0.6524],
          [-0.4214,  0.6513, -0.0150,  ...,  2.2846,  0.1421,  0.6184],
          [ 1.3124,  0.8288,  1.8921,  ..., -0.1697, -0.9722,  0.0097],
          ...,
          [ 1.3404,  0.9747, -1.1471,  ...,  1.6026, -0.1317, -0.8197],
          [-0.1690, -0.8690, -1.7093,  ..., -1.6374, -0.0334, -1.0774],
          [ 2.0206,  0.9985,  1.1270,  ...,  1.2899, -0.3576, -0.8132]]]])
conv1输出的形状： torch.Size([1, 6, 28, 28])
输出层形状 torch.Size([1, 10])
输出层结果： tensor([[-0.1154, -0.1398,  0.1725, -0.0283, -0.0735, -0.2360,  0.1503, -0.1956,
         -0.0787,  0.2216]], grad_fn=<AddmmBackward>)


In [31]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss() 
print("target:", target)

loss = criterion(output, target)
print('loss:',loss)
print('''
反向跟踪loss,使用它的.grad_fn属性,你会看到向下面这样的一个计算图:
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d -> view -> linear -> relu -> linear -> relu -> linear -> MSELoss -> loss''')
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

target: tensor([[ 0.3406,  0.6965, -1.0862, -1.0394,  0.1721,  1.2643, -0.6560, -0.4747,
          1.8006,  1.3724]])
loss: tensor(1.1409, grad_fn=<MseLossBackward>)

反向跟踪loss,使用它的.grad_fn属性,你会看到向下面这样的一个计算图:
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d -> view -> linear -> relu -> linear -> relu -> linear -> MSELoss -> loss


In [32]:
# 优化器
optimizer = torch.optim.SGD(net.parameters(),lr=0.1)

In [33]:
for epoch in range(100):
    optimizer.zero_grad()
    output = net(input)
    loss = criterion(output, target)
    print(loss)
    loss.backward()
    optimizer.step()

tensor(1.1409, grad_fn=<MseLossBackward>)
tensor(0.6371, grad_fn=<MseLossBackward>)
tensor(0.0670, grad_fn=<MseLossBackward>)
tensor(0.0210, grad_fn=<MseLossBackward>)
tensor(0.0683, grad_fn=<MseLossBackward>)
tensor(0.1396, grad_fn=<MseLossBackward>)
tensor(0.4719, grad_fn=<MseLossBackward>)
tensor(0.0028, grad_fn=<MseLossBackward>)
tensor(0.0025, grad_fn=<MseLossBackward>)
tensor(0.0064, grad_fn=<MseLossBackward>)
tensor(0.0142, grad_fn=<MseLossBackward>)
tensor(0.0419, grad_fn=<MseLossBackward>)
tensor(0.0655, grad_fn=<MseLossBackward>)
tensor(0.1972, grad_fn=<MseLossBackward>)
tensor(0.0609, grad_fn=<MseLossBackward>)
tensor(0.1740, grad_fn=<MseLossBackward>)
tensor(0.0595, grad_fn=<MseLossBackward>)
tensor(0.1634, grad_fn=<MseLossBackward>)
tensor(0.0552, grad_fn=<MseLossBackward>)
tensor(0.1478, grad_fn=<MseLossBackward>)
tensor(0.0556, grad_fn=<MseLossBackward>)
tensor(0.1443, grad_fn=<MseLossBackward>)
tensor(0.0512, grad_fn=<MseLossBackward>)
tensor(0.1295, grad_fn=<MseLossBac

In [34]:
torch.save(net, 'net.pth')  # 保存整个网络
torch.save(net.state_dict(), 'net_params.pth')   # 只保存网络中的参数 (速度快, 占内存少)

In [35]:
input2 = torch.ones(1,1,32,32)
print(net(input2))

tensor([[ 0.1206,  0.2151, -0.1010, -0.0846,  0.0959,  0.1254, -0.0207, -0.1024,
          0.2443,  0.3066]], grad_fn=<AddmmBackward>)
